In [13]:
# Importing Libraries
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline

import datetime

import statsmodels.tsa.api as smt
import statsmodels.api as sm
from statsmodels.tools.eval_measures import rmse
import pickle

ModuleNotFoundError: No module named 'sklearn'

### Regressive Modeling

In [7]:
def generate_supervised(data):
    supervised = data.copy()
    
    # Creating column for each lag
    for i in range(1,13):
        col_name = 'lag_' + str(i)
        supervised[col_name] = supervised['sales_diff'].shift(i)
    
    # Dropping null values
    supervised = supervised.dropna().reset_index(drop=True)
    
    supervised.to_csv('../data/input/model.csv', index=False)
    
    return supervised

In [8]:
stationary = pd.read_csv('../data/input/stationary.csv')
model_df = generate_supervised(stationary)

### ARIMA Modeling

In [10]:
def generate_arima_data(data):
    dt_data = data.set_index('orderDate').drop('sales', axis=1)
    dt_data.dropna(axis=0)
    
    dt_data.to_csv('../data/output/arima.csv')
    
    return dt_data

In [11]:
datetime_df = generate_arima_data(stationary)
datetime_df

,Unnamed: 0,sales_diff
orderDate,,
2002-01-10,1,-234.1440
2002-01-11,2,-1088.3000
2002-01-12,3,1436.6320
2002-01-13,4,7199.3870
2002-01-14,5,-7630.7610
...,...,...
2002-09-05,360,3677.6590
2002-09-06,361,-4123.1850
2002-09-07,362,4112.9720


#### Loading the ARIMA Data

In [3]:
def load_data():
    return pd.read_csv('../data/arima_df.csv').set_index('orderDate')

arm_data = load_data()

FileNotFoundError: [Errno 2] No such file or directory: '../data/arima_df.csv'

In [ ]:
arm_data.index = pd.to_datetime(arm_data.index)